In [2]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip uninstall nltk -y
!pip install texthero

Uninstalling nltk-3.2.5:
  Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 1.4MB 6.3MB/s 
     |████████████████████████████████| 245kB 19.3MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp37-none-any.whl size=1434677 sha256=4ab53449f63891f91f98dabc7cdae9ccb1fd0a85fd3f9d0bed7f3399a0ead1a1
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk


In [ ]:
import os

import numpy as np
import pandas as pd

import dill
from sklearn.feature_extraction.text import TfidfVectorizer
import gc
from tqdm import tqdm
import time

import gensim
from gensim.models import Word2Vec
import hashlib
import nltk

import texthero as hero
from texthero import preprocessing


INPUT_DIR = '/content/drive/MyDrive/citation_prediction/input/'

データの読み込み

In [ ]:
all_title_abstract = pd.read_feather(os.path.join(INPUT_DIR, 'all_title_abstract_df.feather'))
all_title_abstract.head()

,title,abstract,cites,clean_title,clean_abstract
0,Heavy-Flavour Production at HERA,We review the theoretical and experimental s...,NaN,heavy flavour production hera,review theoretical experimental status hea...
1,Spectropolarimetric Constraints on the Nature ...,While it is well recognized that interstella...,7.0,spectropolarimetric constraints nature inte...,well recognized interstellar grains made ...
2,A joint analysis of Planck and BICEP2 B modes ...,We analyze BICEP2 and Planck data using a mo...,188.0,joint analysis planck bicep2 b modes includ...,analyze bicep2 planck data using model inc...
3,Molecular movie of ultrafast coherent rotation...,Recording molecular movies on ultrafast time...,8.0,molecular movie ultrafast coherent rotational...,recording molecular movies ultrafast timescal...
4,A Modified Mixed Domain Method for Modeling Ac...,"In this paper, phase correction and amplitud...",NaN,modified mixed domain method modeling acoust...,paper phase correction amplitude compensati...


In [ ]:
# ベクトル化
vec_tfidf = TfidfVectorizer()
title_tfidf = vec_tfidf.fit_transform(all_title_abstract['clean_title'].values)

In [ ]:
start = time.time()

sentences = []
print ("Parsing sentences from training set...")

# Loop over each news article.
for review in tqdm(all_title_abstract['clean_title']):
    try:
        # Split a review into parsed sentences.
        result = review
        h = result.split(" ")
        h = list(filter(("").__ne__, h))
        sentences.append(h)
    except:
        continue

num_features = 200     # Word vector dimensionality
min_word_count = 20   # Minimum word count
num_workers = 1       # Number of threads to run in parallel
context = 10          # Context window size
downsampling = 1e-3   # Downsample setting for frequent words
seed = 42

hashfxn = lambda x: int(hashlib.md5(str(x).encode()).hexdigest(), 16)

print ("Training Word2Vec model...")
# Train Word2Vec model.
model = Word2Vec(sentences, workers=num_workers, hs = 0, sg = 1, negative = 10, iter = 25,\
            size=num_features, min_count = min_word_count, hashfxn=hashfxn,\
            window = context, sample = downsampling, seed=seed)

model_name = str(num_features) + "features_" + str(min_word_count) + "minwords_" \
+ str(context) + "context_len2alldata"
model.init_sims(replace=True)

endmodeltime = time.time()

print ("time : ", endmodeltime-start)

  4%|▍         | 34907/910608 [00:00<00:02, 349066.17it/s]

Parsing sentences from training set...


100%|██████████| 910608/910608 [00:03<00:00, 237818.23it/s]


Training Word2Vec model...
time :  3294.378837108612


In [ ]:
tqdm.pandas()

def des_to_mean_vec(text):
    
    try:
        text = text.strip()
        text_ls = [s for s in text.split(' ') if '' != s]
        return np.mean([model.wv[word] for word in text_ls if word in model.wv.index2word], axis=0)
    except:
        return np.nan

df = all_title_abstract['clean_title'].progress_apply(lambda x: des_to_mean_vec(x))
df.head()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning:

The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version

  0%|          | 905/910608 [00:00<11:55, 1270.71it/s]/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning:

Mean of empty slice.

100%|██████████| 910608/910608 [12:58<00:00, 1169.16it/s]


0    [-0.012905997, 0.03209485, -0.039813425, -0.06...
1    [0.046481848, -0.049888205, 0.059568107, -0.02...
2    [0.022179957, -0.07233812, 0.022202034, 0.0146...
3    [-0.022715338, 0.016935384, -0.0039545572, 0.0...
4    [0.0069463965, -0.018931942, -0.0055342396, 0....
Name: clean_title, dtype: object

In [ ]:
title_df = df.apply(pd.Series)
title_df.columns = ['w2v_title_' + str(col) for col in title_df.columns]
title_df.head()

,w2v_title_0,w2v_title_1,w2v_title_2,w2v_title_3,w2v_title_4,w2v_title_5,w2v_title_6,w2v_title_7,w2v_title_8,w2v_title_9,w2v_title_10,w2v_title_11,w2v_title_12,w2v_title_13,w2v_title_14,w2v_title_15,w2v_title_16,w2v_title_17,w2v_title_18,w2v_title_19,w2v_title_20,w2v_title_21,w2v_title_22,w2v_title_23,w2v_title_24,w2v_title_25,w2v_title_26,w2v_title_27,w2v_title_28,w2v_title_29,w2v_title_30,w2v_title_31,w2v_title_32,w2v_title_33,w2v_title_34,w2v_title_35,w2v_title_36,w2v_title_37,w2v_title_38,w2v_title_39,...,w2v_title_160,w2v_title_161,w2v_title_162,w2v_title_163,w2v_title_164,w2v_title_165,w2v_title_166,w2v_title_167,w2v_title_168,w2v_title_169,w2v_title_170,w2v_title_171,w2v_title_172,w2v_title_173,w2v_title_174,w2v_title_175,w2v_title_176,w2v_title_177,w2v_title_178,w2v_title_179,w2v_title_180,w2v_title_181,w2v_title_182,w2v_title_183,w2v_title_184,w2v_title_185,w2v_title_186,w2v_title_187,w2v_title_188,w2v_title_189,w2v_title_190,w2v_title_191,w2v_title_192,w2v_title_193,w2v_title_194,w2v_title_195,w2v_title_196,w2v_title_197,w2v_title_198,w2v_title_199
0,-0.012906,0.032095,-0.039813,-0.066372,0.017097,-0.022117,-0.080086,0.017539,-0.046864,-0.051337,0.029607,0.022467,-0.015995,-0.001198,0.036831,0.026518,-0.036586,-0.035016,0.059016,-0.036166,-0.012265,0.039259,-0.040893,-0.030775,0.069632,-0.030752,-0.017376,0.018850,0.014894,-0.030017,0.019243,-0.064305,0.006277,0.011914,-0.035515,-0.075505,0.118666,-0.043114,-0.016800,-0.023272,...,-0.004903,-0.011682,-0.079235,0.001925,0.036407,-0.082789,0.027863,-0.065011,0.006397,0.031667,0.009594,-0.129748,0.043917,-0.040884,-0.093635,-0.061318,-0.039132,-0.024082,-0.067468,-0.039793,-0.044412,-0.074770,-0.017522,0.070813,0.021771,0.048754,-0.110685,0.042965,0.028447,-0.050512,0.050352,-0.011993,0.049121,0.000244,0.031563,-0.067616,-0.033102,-0.043941,0.056367,0.075449
1,0.046482,-0.049888,0.059568,-0.021473,-0.059338,-0.084242,-0.023063,0.027394,0.047763,-0.008711,-0.015961,0.018335,0.017503,0.075255,0.062074,0.058488,0.051367,-0.005246,-0.027107,0.059056,-0.009318,0.028276,-0.007149,-0.024733,0.073952,-0.005218,0.038251,-0.032812,0.071093,-0.053634,0.032335,-0.015967,0.023185,-0.095542,-0.009649,-0.009528,0.018077,0.047213,0.026600,0.058996,...,-0.035869,0.005506,-0.100791,0.001448,-0.002738,-0.008396,-0.019204,-0.011945,0.048829,0.089340,0.003413,-0.073785,-0.020939,0.002377,-0.009451,0.008646,-0.006895,-0.083856,0.031254,-0.028420,0.011698,-0.060375,-0.007464,-0.067913,0.073157,0.035453,-0.049106,-0.003096,0.055036,0.001747,0.025692,-0.008794,0.064272,-0.007061,0.034005,0.024788,-0.026774,0.011782,0.012806,0.031553
2,0.022180,-0.072338,0.022202,0.014633,-0.058786,-0.017508,-0.017481,-0.007400,-0.004702,-0.040743,0.049857,-0.007304,0.012228,0.017824,0.028404,-0.020968,0.002031,0.014902,0.016245,0.020874,-0.034717,-0.012334,0.016610,-0.017133,0.037599,-0.021970,0.018060,0.064018,0.051337,-0.035309,0.002117,-0.023648,-0.006148,-0.040359,-0.008217,-0.005930,0.066785,0.002779,0.062434,-0.023239,...,-0.009339,-0.012608,-0.061507,0.006991,0.042691,-0.028944,-0.048203,-0.017672,0.032945,0.057515,0.041344,-0.087346,-0.041469,0.001520,-0.017254,-0.043048,0.070952,-0.018292,-0.055468,-0.039288,0.000061,-0.074036,0.006757,-0.000496,0.082914,0.081040,-0.069138,0.002324,0.015034,0.033043,-0.020493,-0.020279,0.121765,-0.008753,0.052955,0.019510,0.002006,-0.037514,-0.020619,0.057822
3,-0.022715,0.016935,-0.003955,0.008972,-0.041369,-0.068601,-0.040720,0.007750,0.018644,0.022509,-0.021091,-0.042620,-0.031366,-0.001109,0.019274,0.025807,0.040202,0.032206,-0.017049,-0.091817,0.005163,0.006620,0.003240,-0.017990,0.038205,-0.052335,-0.008401,-0.039207,0.088760,-0.046221,0.011616,0.026628,-0.047358,-0.121578,-0.011711,0.043889,-0.039153,-0.036202,0.023576,0.017140,...,-0.005153,0.048073,-0.025132,-0.044124,0.061106,0.003814,0.022246,-0.031199,-0.023093,0.024458,-0.022876,-0.084529,-0.074850,0.018142,0.005739,0.027025,0.072345,-0.046755,-0.000211,0.023911,-0.005689,-0.097176,0.005073,-0.024392,0.048

In [ ]:
title_df.to_feather(os.path.join(INPUT_DIR, 'title_df.feather'))

In [ ]:
del title_df
gc.collect()

53

In [ ]:
file = os.path.join(INPUT_DIR, 'w2v_model_title.dill')
dill.dump(model, open(file,'wb'))